In [1]:
# Sezione 1: Import delle librerie
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Sezione 2: Caricamento dati trading
DATA_PATH = "/home/edocame/Desktop/bollingerBands/DATA"

def load_trading_data():
    """Carica tutti i file CSV di trading dalla cartella DATA_PATH"""
    import os
    data_path = DATA_PATH
    files = [f for f in os.listdir(data_path) if f.lower().endswith('.csv')]
    if not files:
        print(f"Nessun file CSV trovato in {data_path}")
        return {}, None, None
    dfs = {}
    strategy_names = []
    for file in files:
        try:
            with open(os.path.join(data_path, file), 'r', encoding='utf-16') as f:
                lines = f.readlines()
            dates = []
            balances = []
            for line in lines[1:]:
                parts = line.strip().split('\t')
                if len(parts) >= 2:
                    date_str = parts[0].strip()
                    try:
                        balance = float(parts[1].strip())
                        dates.append(date_str)
                        balances.append(balance)
                    except ValueError:
                        continue
            df = pd.DataFrame({'BALANCE': balances}, index=pd.to_datetime(dates))
            df = df.sort_index()
            df = df[~df.index.duplicated(keep='last')]
            strategy_name = file.replace('.csv', '').upper()
            dfs[strategy_name] = df
            strategy_names.append(strategy_name)
        except Exception as e:
            print(f"Errore caricando {file}: {e}")
    combined_df = None
    for name, df in dfs.items():
        if combined_df is None:
            combined_df = df.rename(columns={'BALANCE': name})
        else:
            combined_df = combined_df.join(df.rename(columns={'BALANCE': name}), how='outer')
    combined_df = combined_df.fillna(method='ffill')
    combined_df = combined_df.fillna(method='bfill')
    combined_df = combined_df.resample('D').last().fillna(method='ffill')
    returns_df = combined_df.pct_change().fillna(0)
    for col in returns_df.columns:
        returns_df[col] = np.where(
            (returns_df[col] < -0.5) | (returns_df[col] > 0.5),
            0,
            returns_df[col]
        )
    strategies = {}
    for name in strategy_names:
        strategies[name] = pd.DataFrame({
            'BALANCE': combined_df[name],
            'returns': returns_df[name]
        })
    print(f"\nDataFrame combinato: {combined_df.shape[0]} righe, {combined_df.shape[1]} colonne")
    print(f"L'indice è unico: {returns_df.index.is_unique}")
    return strategies, combined_df, returns_df

print("Caricamento dati...")
strategies_data, combined_df, returns_df = load_trading_data()
print(f"Caricate {len(strategies_data)} strategie: {list(strategies_data.keys())}")

Caricamento dati...

DataFrame combinato: 2380 righe, 28 colonne
L'indice è unico: True
Caricate 28 strategie: ['AUDJPY_14D_1', 'EURGBP_14D_1', 'AUDNZD_14D_1', 'USDCHF_1440_01', 'EURGBP_7200_02', 'EURUSD_7200_02', 'GBPUSD_1440_01', 'USDCAD_1440_01', 'GBPUSD_7200_02', 'GBPCHF_1440_01', 'EURUSD_1440_01', 'GBPUSD_14D_1', 'GBPCHF_7200_02', 'GBPCHF_14D_1', 'AUDJPY_1440_01', 'AUDJPY_7200_02', 'EURCHF_14D_1', 'EURCHF_1440_01', 'USDCHF_7200_02', 'EURUSD_14D_1', 'NZDJPY_7200_02', 'USDCAD_7200_02', 'USDCAD_14D_1', 'EURCHF_7200_02', 'USDCHF_14D_1', 'AUDNZD_7200_02', 'EURGBP_1440_01', 'AUDNZD_1440_01']

DataFrame combinato: 2380 righe, 28 colonne
L'indice è unico: True
Caricate 28 strategie: ['AUDJPY_14D_1', 'EURGBP_14D_1', 'AUDNZD_14D_1', 'USDCHF_1440_01', 'EURGBP_7200_02', 'EURUSD_7200_02', 'GBPUSD_1440_01', 'USDCAD_1440_01', 'GBPUSD_7200_02', 'GBPCHF_1440_01', 'EURUSD_1440_01', 'GBPUSD_14D_1', 'GBPCHF_7200_02', 'GBPCHF_14D_1', 'AUDJPY_1440_01', 'AUDJPY_7200_02', 'EURCHF_14D_1', 'EURCHF_1440_01'

# Sezione 3: Calcolo e Visualizzazione della Matrice di Correlazione
Questa sezione calcola la matrice di correlazione tra i rendimenti delle strategie, mostra statistiche riassuntive, identifica le coppie altamente correlate e visualizza i risultati con heatmap e istogrammi.

In [3]:
# Calcolo matrice di correlazione e visualizzazione
correlation_matrix = returns_df.corr()
corr_values = correlation_matrix.values
upper_triangle = np.triu(corr_values, k=1)
non_zero_corr = upper_triangle[upper_triangle != 0]
print(f"📈 STATISTICHE CORRELAZIONE:")
print(f"   Correlazione media: {np.mean(non_zero_corr):.3f}")
print(f"   Correlazione mediana: {np.median(non_zero_corr):.3f}")
print(f"   Correlazione min: {np.min(non_zero_corr):.3f}")
print(f"   Correlazione max: {np.max(non_zero_corr):.3f}")
print(f"   Deviazione standard: {np.std(non_zero_corr):.3f}")
high_corr_threshold = 0.8
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if abs(corr_value) > high_corr_threshold:
            high_corr_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j], corr_value))
print(f"\n🚨 COPPIE AD ALTA CORRELAZIONE (|r| > {high_corr_threshold}):")
if high_corr_pairs:
    for pair1, pair2, corr in high_corr_pairs:
        print(f"   {pair1} ↔ {pair2}: {corr:.3f}")
else:
    print("   Nessuna coppia con correlazione superiore alla soglia")
fig_corr = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Matrice di Correlazione", "Distribuzione Correlazioni"),
    specs=[[{"type": "heatmap"}, {"type": "histogram"}]],
    horizontal_spacing=0.1
)
fig_corr.add_trace(
    go.Heatmap(
        z=correlation_matrix.values,
        x=correlation_matrix.columns,
        y=correlation_matrix.columns,
        colorscale='RdBu',
        zmid=0,
        text=np.round(correlation_matrix.values, 2),
        texttemplate="%{text}",
        textfont={"size": 8},
        colorbar=dict(title="Correlazione", x=0.45)
    ),
    row=1, col=1
)
fig_corr.add_trace(
    go.Histogram(
        x=non_zero_corr,
        nbinsx=30,
        name="Distribuzione Correlazioni",
        marker=dict(color='skyblue', opacity=0.7)
    ),
    row=1, col=2
)
fig_corr.add_vline(
    x=np.mean(non_zero_corr), 
    line_dash="dash", 
    line_color="red",
    annotation_text=f"Media: {np.mean(non_zero_corr):.3f}",
    row=1, col=2
)
fig_corr.add_vline(
    x=np.median(non_zero_corr), 
    line_dash="dash", 
    line_color="green",
    annotation_text=f"Mediana: {np.median(non_zero_corr):.3f}",
    row=1, col=2
)
fig_corr.update_layout(
    title="Analisi Correlazione tra Strategie di Trading",
    height=600,
    showlegend=False
)
fig_corr.update_xaxes(title_text="Correlazione", row=1, col=2)
fig_corr.update_yaxes(title_text="Frequenza", row=1, col=2)
fig_corr.show()
mean_corr_per_strategy = correlation_matrix.mean().sort_values(ascending=False)
print(f"\n📊 STRATEGIE CON CORRELAZIONE MEDIA PIÙ ALTA:")
for strategy, avg_corr in mean_corr_per_strategy.head(5).items():
    print(f"   {strategy}: {avg_corr:.3f}")
print(f"\n📊 STRATEGIE CON CORRELAZIONE MEDIA PIÙ BASSA:")
for strategy, avg_corr in mean_corr_per_strategy.tail(5).items():
    print(f"   {strategy}: {avg_corr:.3f}")

📈 STATISTICHE CORRELAZIONE:
   Correlazione media: 0.061
   Correlazione mediana: 0.034
   Correlazione min: -0.085
   Correlazione max: 0.461
   Deviazione standard: 0.089

🚨 COPPIE AD ALTA CORRELAZIONE (|r| > 0.8):
   Nessuna coppia con correlazione superiore alla soglia



📊 STRATEGIE CON CORRELAZIONE MEDIA PIÙ ALTA:
   GBPCHF_1440_01: 0.139
   GBPCHF_7200_02: 0.129
   GBPCHF_14D_1: 0.126
   GBPUSD_1440_01: 0.121
   GBPUSD_7200_02: 0.118

📊 STRATEGIE CON CORRELAZIONE MEDIA PIÙ BASSA:
   USDCAD_1440_01: 0.076
   AUDNZD_1440_01: 0.073
   USDCAD_14D_1: 0.068
   AUDNZD_7200_02: 0.063
   AUDNZD_14D_1: 0.056


# Sezione 4: Analisi delle Componenti Principali (PCA)
Questa sezione esegue la PCA sui rendimenti delle strategie, analizza la varianza spiegata, visualizza scree plot e biplot, e interpreta il livello di collinearità.

In [4]:
# Analisi PCA
scaler = StandardScaler()
returns_scaled = scaler.fit_transform(returns_df.fillna(0))
pca = PCA()
pca_result = pca.fit_transform(returns_scaled)
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
n_components_90 = np.argmax(cumulative_variance >= 0.90) + 1
n_components_80 = np.argmax(cumulative_variance >= 0.80) + 1
print(f"📊 RISULTATI PCA:")
print(f"   Componenti totali: {len(explained_variance)}")
print(f"   Componenti per 80% varianza: {n_components_80}")
print(f"   Componenti per 90% varianza: {n_components_90}")
print(f"   Componenti per 95% varianza: {n_components_95}")
print(f"   Riduzione dimensionalità: {((len(explained_variance) - n_components_95) / len(explained_variance)) * 100:.1f}%")
n_top_components = min(5, len(explained_variance))
print(f"\n🎯 TOP {n_top_components} COMPONENTI PRINCIPALI:")
for i in range(n_top_components):
    print(f"\n   Componente {i+1} (spiega {explained_variance[i]:.1%} della varianza):")
    loadings = pca.components_[i]
    top_strategies_idx = np.argsort(np.abs(loadings))[::-1][:3]
    for idx in top_strategies_idx:
        strategy = returns_df.columns[idx]
        loading = loadings[idx]
        print(f"     {strategy}: {loading:.3f}")
fig_pca = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Varianza Spiegata per Componente", "Varianza Cumulativa",
                   "Scree Plot", "Biplot PC1 vs PC2"),
    specs=[[{"type": "bar"}, {"type": "scatter"}],
           [{"type": "scatter"}, {"type": "scatter"}]],
    vertical_spacing=0.1,
    horizontal_spacing=0.1
)
fig_pca.add_trace(
    go.Bar(
        x=list(range(1, min(21, len(explained_variance)+1))),
        y=explained_variance[:20],
        name="Varianza Spiegata",
        marker=dict(color='skyblue')
    ),
    row=1, col=1
)
fig_pca.add_trace(
    go.Scatter(
        x=list(range(1, min(21, len(cumulative_variance)+1))),
        y=cumulative_variance[:20],
        mode='lines+markers',
        name="Varianza Cumulativa",
        line=dict(color='red', width=2)
    ),
    row=1, col=2
)
for threshold, color in [(0.8, 'orange'), (0.9, 'green'), (0.95, 'purple')]:
    fig_pca.add_hline(
        y=threshold, 
        line_dash="dash", 
        line_color=color,
        annotation_text=f"{threshold*100:.0f}%",
        row=1, col=2
    )
fig_pca.add_trace(
    go.Scatter(
        x=list(range(1, min(21, len(explained_variance)+1))),
        y=explained_variance[:20],
        mode='lines+markers',
        name="Scree Plot",
        line=dict(color='darkgreen', width=2)
    ),
    row=2, col=1
)
pc1_loadings = pca.components_[0]
pc2_loadings = pca.components_[1]
fig_pca.add_trace(
    go.Scatter(
        x=pc1_loadings,
        y=pc2_loadings,
        mode='markers+text',
        text=returns_df.columns,
        textposition="top center",
        name="Strategie (PC1 vs PC2)",
        marker=dict(size=8, color='red')
    ),
    row=2, col=2
)
for i, strategy in enumerate(returns_df.columns):
    if abs(pc1_loadings[i]) > 0.1 or abs(pc2_loadings[i]) > 0.1:
        fig_pca.add_annotation(
            x=pc1_loadings[i], y=pc2_loadings[i],
            ax=0, ay=0,
            arrowhead=2, arrowsize=1, arrowwidth=2,
            arrowcolor="blue",
            row=2, col=2
        )
fig_pca.update_layout(
    title="Analisi delle Componenti Principali (PCA)",
    height=800,
    showlegend=False
)
fig_pca.update_xaxes(title_text="Componente", row=1, col=1)
fig_pca.update_yaxes(title_text="Varianza Spiegata", row=1, col=1)
fig_pca.update_xaxes(title_text="Componente", row=1, col=2)
fig_pca.update_yaxes(title_text="Varianza Cumulativa", row=1, col=2)
fig_pca.update_xaxes(title_text="Componente", row=2, col=1)
fig_pca.update_yaxes(title_text="Varianza Spiegata", row=2, col=1)
fig_pca.update_xaxes(title_text=f"PC1 ({explained_variance[0]:.1%})", row=2, col=2)
fig_pca.update_yaxes(title_text=f"PC2 ({explained_variance[1]:.1%})", row=2, col=2)
fig_pca.show()
if n_components_95 < len(explained_variance) * 0.5:
    print(f"   🔴 ALTA COLLINEARITÀ: Solo {n_components_95} componenti spiegano il 95% della varianza")
    print(f"   → Le strategie sono altamente correlate e ridondanti")
elif n_components_95 < len(explained_variance) * 0.8:
    print(f"   🟡 MEDIA COLLINEARITÀ: {n_components_95} componenti spiegano il 95% della varianza")
    print(f"   → Esiste una moderata correlazione tra le strategie")
else:
    print(f"   🟢 BASSA COLLINEARITÀ: {n_components_95} componenti spiegano il 95% della varianza")
    print(f"   → Le strategie sono relativamente indipendenti")

📊 RISULTATI PCA:
   Componenti totali: 28
   Componenti per 80% varianza: 18
   Componenti per 90% varianza: 23
   Componenti per 95% varianza: 25
   Riduzione dimensionalità: 10.7%

🎯 TOP 5 COMPONENTI PRINCIPALI:

   Componente 1 (spiega 10.4% della varianza):
     GBPCHF_1440_01: 0.330
     GBPCHF_7200_02: 0.318
     GBPCHF_14D_1: 0.303

   Componente 2 (spiega 7.1% della varianza):
     GBPCHF_14D_1: 0.337
     EURGBP_14D_1: 0.308
     USDCAD_7200_02: -0.295

   Componente 3 (spiega 6.6% della varianza):
     AUDJPY_7200_02: 0.443
     NZDJPY_7200_02: 0.345
     AUDNZD_7200_02: 0.334

   Componente 4 (spiega 5.7% della varianza):
     EURCHF_14D_1: 0.391
     EURCHF_7200_02: 0.351
     EURGBP_1440_01: -0.325

   Componente 5 (spiega 5.4% della varianza):
     EURCHF_1440_01: 0.333
     USDCAD_14D_1: -0.319
     EURCHF_7200_02: 0.317


   🟢 BASSA COLLINEARITÀ: 25 componenti spiegano il 95% della varianza
   → Le strategie sono relativamente indipendenti
